In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

In [3]:
from wordvecs import WordVectors, EmbeddingLayer

wordvectors = WordVectors(fname="/data/matthew/GoogleNews-vectors-negative300.bin", negvectors=False)

In [4]:
from sentiment_sents import Sentiment

# just load the sentences from the CNN system
sentiment = Sentiment("prevwork/CNN_sentence/")

In [5]:
#from stanfordSentimentReader import StanfordSentimentReader

#sentiment = StanfordSentimentReader('/data/matthew/stanfordSentimentTreebank')

In [6]:
#len(sentiment.test_X), len(sentiment.train_X), len(sentiment.dev_X)

In [7]:
len(wordvectors.vectors)

3000000

In [8]:
# make the labels binary since that is what we are currently targetting in the training
#sentiment.train_Y = (np.array(sentiment.train_Y) > .6).astype('int32')
#sentiment.test_Y = (np.array(sentiment.test_Y) > .6).astype('int32')
#sentiment.dev_Y = (np.array(sentiment.dev_Y) > .6).astype('int32')

In [9]:
#len(sentiment.test_Y), len(sentiment.train_Y), len(sentiment.dev_Y)

In [10]:
for w in sentiment.test_X:
    wordvectors.tokenize(w)
for w in sentiment.train_X:
    wordvectors.tokenize(w)
#for w in sentiment.dev_X:
#    wordvectors.tokenize(w)

In [11]:
len(wordvectors.word_location)

21418

In [59]:
class SentimentExp(object):
    
    def __init__(self, train_X, train_Y, wordvecs=wordvectors):
        self.train_X = train_X
        self.train_Y = train_Y
        self.wordvecs = wordvecs
        
        self.input_size = 10  # not used
        self.batch_size = 50
        
        self.learning_rate = .01
        self.momentum = .9
        
        #self.train_X_rep = np.array([[self.getRep(x)] for x in self.train_X])
        self.train_X_rep = np.array([wordvecs.tokenize(x) for x in self.train_X])
        
        self._setup()

    def getRep(self, sent):
        ret = []
        for i in xrange(self.input_size):
            if i < len(sent):
                ret.append(self.wordvecs[sent[i]])
            else:
                ret.append(np.zeros(self.wordvecs.vector_size))
        return np.matrix(ret).reshape((1, self.input_size, self.wordvecs.vector_size))

    def _setup(self):
        self.x_batch = T.imatrix('x')
        self.y_batch = T.ivector('y')
        
        self.input_l = lasagne.layers.InputLayer((None, 50))
        
        self.embedding_l = EmbeddingLayer(
            self.input_l,
            W=self.wordvecs.get_numpy_matrix(),
            add_word_params=False,
        ) 
        
        self.first_l = lasagne.layers.Conv2DLayer(
            self.embedding_l,
            num_filters=1,
            filter_size=(2, self.wordvecs.vector_size),
            name='conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.first_l_max = lasagne.layers.MaxPool2DLayer(
            self.first_l,
            name='maxing1',
            pool_size=(1,1)  # the number 9 should be 50-1 since that would mean it maxes over the whole input....
        )
        
        self.hidden1_l = lasagne.layers.DenseLayer(
            self.first_l_max,
            num_units=50,
            name='dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.hidden1_l_drop = lasagne.layers.DropoutLayer(
            self.hidden1_l,
            name='drop1',
            p=.25,
        )
        
        self.out_l = lasagne.layers.DenseLayer(
            self.hidden1_l_drop,
            num_units=1,
            name='dens2',
            nonlinearity=lasagne.nonlinearities.tanh,
        )
        
        self.output = lasagne.layers.get_output(self.out_l, self.x_batch)
        
        self.loss_vec_old = (self.output.reshape((self.output.size,)) - self.y_batch) ** 2
        self.output_diff = T.neq((self.output.flatten() > .5),(self.y_batch > .5)).sum()
        self.loss_vec = lasagne.objectives.binary_crossentropy(T.clip(self.output.reshape((self.output.size,)), .01, .99), self.y_batch)
        
        self.all_params = lasagne.layers.get_all_params(self.out_l)
        
        self.updates = lasagne.updates.adagrad(self.loss_vec.mean(), self.all_params, .01)
        #self.updates = lasagne.updates.apply_momentum(self.updates_adagrad)
        
        self.train_func = theano.function(
            [self.x_batch, self.y_batch],
            [self.loss_vec.mean(), self.loss_vec],
            updates=self.updates,
        )
        
        self.loss_func = theano.function(
            [self.x_batch, self.y_batch],
            [self.loss_vec.sum(), self.loss_vec, self.output_diff],
        )
        
    def _make_zero(self):
        self.embedding_l.W.get_value(borrow=True)[0,:] = 0
        
    def train(self):
        for s in xrange(0, len(self.train_X_rep), self.batch_size):
            end = s + self.batch_size
            if end > len(self.train_X_rep):
                end = len(self.train_X_rep)
            X_vals = np.array(self.train_X_rep[s:end]).astype('int32')
            y_vals = np.array(self.train_Y[s:end]).astype('int32')
            loss, _ = self.train_func(X_vals, y_vals)
            self._make_zero()
            
    def test_loss(self, test_X, test_Y):
        test_X_rep = np.array([self.wordvecs.tokenize(x) for x in test_X])
        loss_sum = 0.0
        wrong = 0.0
        for s in xrange(0, len(test_X_rep), self.batch_size):
            end = s + self.batch_size
            if end > len(self.train_X_rep):
                end = len(self.train_X_rep)
            X_vals = np.array(self.train_X_rep[s:end]).astype('int32')
            y_vals = np.array(self.train_Y[s:end]).astype('int32')
            loss, _, output_diff = self.loss_func(X_vals, y_vals)
            wrong += output_diff
            loss_sum += loss
        return loss_sum / len(test_X_rep), wrong / len(test_X_rep)
    
experiment = SentimentExp(sentiment.train_X, sentiment.train_Y)

In [13]:
experiment.all_params

[conv1.W, conv1.b, dens1.W, dens1.b, dens2.W, dens2.b]

In [14]:
for i in xrange(300):
    if i % 15 == 0:
        print i, experiment.test_loss(sentiment.train_X, sentiment.train_Y), experiment.test_loss(sentiment.test_X, sentiment.test_Y)
    experiment.train()

0 (2.1525695871550785, 0.50164126611957793) (2.0915951507427284, 0.48686679174484054)
15 (2.3151520754462234, 0.50164126611957793) (2.2473464477132898, 0.48686679174484054)
30 (2.3151520754462234, 0.50164126611957793) (2.2473464477132898, 0.48686679174484054)
45 (2.3151520754462234, 0.50164126611957793) (2.2473464477132898, 0.48686679174484054)
60 (2.3151520754462234, 0.50164126611957793) (2.2473464477132898, 0.48686679174484054)


KeyboardInterrupt: 

In [ ]:
experiment.train_X_rep[1,:]

In [ ]:
experiment.train_Y[1]

In [ ]:
experiment.test_loss(sentiment.train_X, sentiment.train_Y)

In [ ]:
experiment.test_loss(sentiment.dev_X, sentiment.dev_Y)

In [ ]:
experiment.test_loss(sentiment.test_X, sentiment.test_Y)

In [18]:
experiment.first_l.b.get_value(borrow=True)

array([ -6.04371018e-03,   1.78176286e-02,   1.09569314e-02,
         1.67699890e-02,   1.99813069e-02,   1.18091645e-03,
        -7.71580864e-03,  -7.66937027e-03,   2.58295118e-03,
         1.91439796e-02,  -4.52293052e-03,   9.89952002e-03,
         1.27791829e-02,   1.08079915e-02,  -6.18459987e-03,
         8.54377781e-03,   1.81448576e-03,  -1.12456523e-02,
        -1.39540722e-02,  -9.03013292e-03,  -1.47806321e-02,
         1.93713232e-02,   2.06622801e-03,  -1.36212534e-02,
         1.20468446e-02,  -9.08213372e-03,   1.39999114e-02,
         1.24535194e-02,   1.33896294e-02,   1.07211256e-02,
         9.71938456e-03,   6.09554924e-03,  -7.61628915e-03,
        -5.02340545e-03,   1.07865300e-02,  -1.67957726e-02,
        -1.98590815e-02,   7.27865640e-03,  -1.21863609e-02,
        -1.78227560e-02,   1.00208855e-02,  -1.13455128e-02,
         8.78016649e-03,  -5.12723489e-03,  -2.32765902e-03,
        -1.07733866e-02,  -1.86382091e-03,  -6.29751929e-03,
         9.08809137e-03,

In [ ]:
experiment.

In [62]:
test_func2 = theano.function(
    [experiment.x_batch],
    [
        lasagne.layers.get_output(experiment.first_l, experiment.x_batch).shape,
        lasagne.layers.get_output(experiment.first_l_max, experiment.x_batch).shape
    ]
)

In [63]:
test_func2(np.array(experiment.train_X_rep[0:1]).astype('int32'))

[array([ 1,  1, 49,  1]), array([ 1,  1, 49,  1])]

In [15]:
test_func = theano.function(
    [experiment.x_batch, experiment.y_batch],
    [experiment.loss_vec.mean(), experiment.loss_vec, experiment.output, 
     T.grad(experiment.loss_vec.mean(), experiment.out_l.get_params()[0]),
     experiment.out_l.get_params()[0], experiment.y_batch, 
     #lasagne.layers.get_output(experiment.first_l, experiment.x_batch)
    ]
)

In [16]:
test_func(np.array(experiment.train_X_rep[0:50]).astype('int32'),np.array(experiment.train_Y[0:50]).astype('int32'))

[array(2.215707863918104),
 array([ 0.01005034,  0.01005034,  0.01005034,  0.01005034,  0.01005034,
         4.60517019,  4.60517019,  0.01005034,  4.60517019,  0.01005034,
         0.01005034,  4.60517019,  4.60517019,  4.60517019,  4.60517019,
         0.01005034,  0.01005034,  4.60517019,  0.01005034,  0.01005034,
         0.01005034,  0.01005034,  0.01005034,  0.01005034,  0.01005034,
         0.01005034,  4.60517019,  4.60517019,  4.60517019,  0.01005034,
         4.60517019,  4.60517019,  4.60517019,  0.01005034,  0.01005034,
         4.60517019,  0.01005034,  4.60517019,  0.01005034,  4.60517019,
         4.60517019,  4.60517019,  4.60517019,  0.01005034,  0.01005034,
         4.60517019,  4.60517019,  0.01005034,  4.60517019,  4.60517019]),
 array([[-0.99999746],
        [-0.99999921],
        [-0.99999755],
        [-0.99999937],
        [-0.99997959],
        [-0.99998647],
        [-0.97755117],
        [-0.99999905],
        [-0.99999561],
        [-0.99999844],
        [-0

In [ ]:
gg_k = theano.shared(0.)
gg_i = T.iscalar('x')

gg_res, gg_update = theano.scan(lambda: {gg_k:(gg_k + 1)}, sequences=[range(gg_i)])

In [ ]:
gg_update

In [ ]:
a1 = T.matrix()
f1 = theano.function([a1], [a1.shape])

In [ ]:
t1 = np.array([[1,2,3,0,0],[4,5,6,7,0],[2,2,2,2,2]])

In [ ]:
t1.dtype

In [ ]:
f1(t1)

In [ ]:
%reload_ext wordvecs

In [ ]:
np.zeros(10)